# Практическое задание

## Данные о студенте

1. **ФИО**: Ковалев Виктор Васильевич
2. **Факультет**: МехМат
3. **Курс**: 1, магистратура
4. **Группа**: М1

## Замечания

* Заполненный ноутбук необходимо сдать боту
* Соблюдаем кодекс чести (по нулям и списавшему, и давшему списать)
* Можно (и нужно!) применять для реализации только библиотеку **Numpy**
* Ничего, крому Numpy, нельзя использовать для реализации
* **Keras** используется только для тестирования Вашей реализации
* Если какой-то из классов не проходит приведенные тесты, то соответствующее задание не оценивается
* Возможно использование дополнительных (приватных) тестов


**Примечание: задание делалось в google colab, поэтому немного были изменены импорты  из keras. Поэтому в jupiter notebook может работать не корректно**

## Реализация собственного нейросетевого пакета для запуска и обучения нейронных сетей

Задание состоит из трёх частей:
1. Реализация прямого вывода нейронной сети (5 баллов)
2. Реализация градиентов по входу и распространения градиента по сети (5 баллов)
3. Реализация градиентов по параметрам и метода обратного распространения ошибки с обновлением парметров сети (10 баллов)

Дополнительные баллы можно получить при реализации обучения сети со свёрточными слоями (10 баллов), с транспонированной свёрткой (10 баллов), дополнительного оптимизатора (5 баллов).

###  1. Реализация вывода собственной нейронной сети

1.1 Внимательно ознакомьтесь с интерфейсом слоя. Любой слой должен содержать как минимум три метода:
- конструктор
- прямой вывод
- обратный вывод, производные по входу и по параметрам

In [1]:
class Layer(object):
    def __init__(self):
        self.name = 'Layer'
    def forward(self, input_data):
        pass
    def backward(self, input_data):
        return [self.grad_x(input_data), self.grad_param(input_data)]

    def grad_x(self, input_data):
        pass
    def grad_param(self, input_data):
        return []

    def update_param(self, grads, learning_rate):
        pass


1.2 Ниже предствален интерфейс класса  Network. Обратите внимание на реализацию метода predict, который последовательно обрабатывает входные данные слой за слоем.

In [40]:
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

class Network(object):
    def __init__(self, layers, loss=None):
        self.name = 'Network'
        self.layers = layers
        self.loss = loss

    def forward(self, input_data):
        return self.predict(input_data)

    def grad_x(self, input_data, labels):
        inputs=input_data
        my_list=[]
        for layer in self.layers:
            my_list.append(inputs)
            inputs=layer.forward(inputs)
        out=self.loss.grad_x(inputs,labels)
        inputs_list=my_list[::-1]
        for i,layer in enumerate(self.layers[::-1]):
            if layer.name == 'Softmax':
                output_softmax=layer.forward(inputs_list[i])
                out= output_softmax-labels
            elif layer.name == 'Dense':
                dense_list=[]
                for data in out:
                    # print(data.shape,layer.W.T.shape)
                    dense_list.append(data @ layer.W.T)
                out=np.array(dense_list)
                # out=out.sum(axis=0)
            elif layer.name == 'ReLU':
                relu_list=[]
                for j,data in enumerate(out):
                    relu_list.append(data  @ layer.grad_x(inputs_list[i])[j].T)
                out=np.array(relu_list)
                # out=out.sum(axis=0)
            else:
                pass
        return out.sum(axis=0)
    def grad_param(self, input_data, labels):
        inputs=input_data
        grad_list=[]
        my_list=[]
        for layer in self.layers:
            my_list.append(inputs)
            inputs=layer.forward(inputs)
        out=self.loss.grad_x(inputs,labels)
        inputs_list=my_list[::-1]
        for i,layer in enumerate(self.layers[::-1]):
            if layer.name == 'Softmax':
                output_softmax=layer.forward(inputs_list[i])
                out= output_softmax-labels
                grad_list.append(0)
                # print(out.shape)
            elif layer.name == 'Dense':
                w_list=[]
                for j,data in enumerate(out):
                    w_list.append(data @ layer.grad_W(inputs_list[i])[j])
                grad_list.append([np.array(w_list),out])
                # out=out @ layer.W.T
                dense_list=[]
                for data in out:
                    # print(data.shape,layer.W.T.shape)
                    dense_list.append(data @ layer.W.T)
                out=np.array(dense_list)
                # print(out.shape)
            elif layer.name == 'ReLU':
                grad_list.append(0)
                relu_list=[]
                for j,data in enumerate(out):
                    # print(data,layer.grad_x(inputs_list[i])[j])
                    relu_list.append(data  @ layer.grad_x(inputs_list[i])[j])
                out=np.array(relu_list)
                # print(out.shape)
            else:
                pass
        return grad_list
    def update(self, grad_list, learning_rate):
        for i,layer in enumerate(self.layers[::-1]):
            if layer.name=='Dense':
                grad_w,grad_b=grad_list[i]
                layer.update_W(grad_w ,learning_rate)
                layer.update_b(grad_b ,learning_rate)
    def predict(self, input_data):
        current_input = input_data
        for layer in self.layers:
            current_input = layer.forward(current_input)
        return current_input

    def calculate_loss(self, input_data, labels):
        return self.loss.forward(self.predict(input_data), labels)

    def train_step(self, input_data, labels, learning_rate=0.001):
        grad_list = self.grad_param(input_data, labels)
        self.update(grad_list, learning_rate)

    def fit(self, trainX, trainY, validation_split=0.25,
            batch_size=1, nb_epoch=1, learning_rate=0.01):

        train_x, val_x, train_y, val_y = train_test_split(trainX, trainY,
                                                          test_size=validation_split,
                                                          random_state=42)
        for epoch in range(nb_epoch):
            #train one epoch
            for i in tqdm(range(int(len(train_x)/batch_size))):
                batch_x = train_x[i*batch_size: (i+1)*batch_size]
                batch_y = train_y[i*batch_size: (i+1)*batch_size]
                self.train_step(batch_x, batch_y, learning_rate)
            #validate
            val_accuracy = self.evaluate(val_x, val_y)
            print('%d epoch: val %.2f' %(epoch+1, val_accuracy))

    def evaluate(self, testX, testY):
        y_pred = np.argmax(self.predict(testX), axis=1)
        y_true = np.argmax(testY, axis=1)
        val_accuracy = np.sum((y_pred == y_true))/(len(y_true))
        return val_accuracy

#### 1.1 Необходимо реализовать метод forward для вычисления следующих слоёв:

- DenseLayer
- ReLU
- Softmax
- FlattenLayer
- MaxPooling

In [3]:
#импорты
import numpy as np

In [4]:
class DenseLayer(Layer):
    def __init__(self, input_dim, output_dim, W_init=None, b_init=None):
        self.name = 'Dense'
        self.input_dim = input_dim
        self.output_dim = output_dim
        if W_init is None:
            # self.W = np.random.randn(input_dim, output_dim)
            self.W =np.eye(input_dim, output_dim)
        else:
            self.W = W_init
        if b_init is None:
            self.b = np.zeros(output_dim)
        else:
            self.b = b_init
    def forward(self, input_data):
        out=np.empty((input_data.shape[0],self.output_dim))
        # print(self.W.shape,input_data.shape)
        for i,mat in enumerate(input_data):
            my_mat= mat @ self.W + self.b
            # print(self.W.shape)
            out[i]=my_mat
        # print(np.mean(out))
        return out
    def grad_x(self, input_data):
        n=input_data.shape[0]
        my_shape=(n,*self.W.T.shape)
        out=np.empty(my_shape)
        for i in range(n):
            out[i]=self.W.T
        return out
    def grad_b(self, input_data):
        my_list=[]
        for i in range(input_data.shape[0]):
            my_list.append(np.eye(self.b.shape[-1],self.b.shape[-1]))
        return np.array(my_list)

    def grad_W(self, input_data):
        my_list=[]
        n=self.output_dim
        for x in input_data:
            my_matrix=[]
            for i in range(n):
                my_vektor=np.zeros((self.input_dim,n)).T
                # print(my_vektor.shape)
                my_vektor[i]=x
                my_matrix.append(my_vektor.T.flatten())
            my_list.append(my_matrix)
        return np.array(my_list)


    def update_W(self, grad, learning_rate):
        self.W -= learning_rate * np.mean(grad, axis=0).reshape(self.W.shape)

    def update_b(self, grad,  learning_rate):
        self.b -= learning_rate * np.mean(grad, axis=0).reshape(self.b.shape)

    def update_param(self, params_grad, learning_rate):
        self.update_W(params_grad[0], learning_rate)
        self.update_b(params_grad[1], learning_rate)

    def grad_param(self, input_data):
        return [self.grad_W(input_data), self.grad_b(input_data)]

class ReLU(Layer):
    def __init__(self):
        self.name = 'ReLU'
    def forward(self, input_data):
        return np.array([[max(0,x) for x in data]for data in input_data])
    def grad_x(self, input_data):
        my_shape=(input_data.shape[0],input_data.shape[1],input_data.shape[1])
        out=np.zeros(my_shape)
        my_arr=self.forward(input_data)
        for i in range(my_shape[0]):
            for j in range(my_shape[1]):
                if my_arr[i,j]>0:
                    out[i,j,j]=1
        return out

class Softmax(Layer):
    def __init__(self):
        self.name = 'Softmax'
    def forward(self, input_data):
        out=np.empty(input_data.shape)
        for i,data in enumerate(input_data):
            exp_values = np.exp(data - np.max(data))
            out[i]=exp_values/sum(exp_values)
        return out
    def grad_x(self, input_data):
        my_shape=(input_data.shape[0],input_data.shape[1],input_data.shape[1])
        out=np.empty(my_shape)
        soft_arr=self.forward(input_data)
        for i in range(my_shape[0]):
            for j in range(my_shape[1]):
                for k in range(j,my_shape[2]):
                  if j==k:
                      out[i,j,k]=soft_arr[i][j]*(1-soft_arr[i][j])
                  else:
                      out[i,j,k]=-soft_arr[i][j]*soft_arr[i][k]
                      out[i,k,j]=-soft_arr[i][j]*soft_arr[i][k]
        return out




class FlattenLayer(Layer):
    def __init__(self):
        self.name = 'Flatten'

    def forward(self, input_data):
        # print(input_data.shape)
        num,c,h,w=input_data.shape
        out=np.empty((num,c*h*w))
        for i,matrix in enumerate(input_data):
            out[i]=matrix.flatten()

        return out
    def grad_x(self):
        pass
class MaxPooling(Layer):
    def __init__(self,kernel_size=2):
        self.name = 'MaxPool'
        self.ker_size=kernel_size
    def forward(self,input_data):
        batch,chenal,h,w=input_data.shape
        x_stop=w
        y_stop=h
        stride=self.ker_size
        out=np.empty((batch,chenal,h//self.ker_size,w//self.ker_size))
        for n_bat in range(batch):
            for n_chenal in range(chenal):
                for i in range(0,y_stop,self.ker_size):
                    for j in range(0,x_stop,self.ker_size):
                        my_max=np.max(input_data[n_bat,n_chenal,i:i+self.ker_size,j:j+self.ker_size])
                        out[n_bat,n_chenal,i//self.ker_size,j//self.ker_size]=my_max
        return out
    def grad_x(self):
        pass

#### 1.2 Реализуйте теперь свёрточный слой и транспонированную свёртку  (опционально)

In [5]:
class Conv2DLayer(Layer):
    def __init__(self, kernel_size=3, input_channels=2, output_channels=3,
                 padding='same', stride=1, K_init=None, b_init=None):
        # padding: 'same' или 'valid'
        # Работаем с квадратными ядрами, поэтому kernel_size - одно число
        # Работаем с единообразным сдвигом, поэтому stride - одно число
        # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
        self.name = 'Conv2D'
        self.kernel_size = kernel_size
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.kernel = K_init
        self.bias = b_init
        self.padding = padding
        self.stride = stride
    def forward(self, input_data):
        # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
        # Вначале нужно проверить на согласование размерностей входных данных и ядра!
        # Нужно заполнить Numpy-тензор out
        #свертка скопирова из предыдущего дз
        if self.kernel is None:
            ker=np.eye((self.kernel_size,self.kernel_size))
        else:
            ker=self.kernel
        if self.padding=='valid':
            _,_,h,w=input_data.shape
            start_y=self.kernel_size//2
            stop_y=h-self.kernel_size//2
            start_x=self.kernel_size//2
            stop_x=w-self.kernel_size//2
            my_list=[]
            def conv(num,c,i,j,my_kernel=self.kernel,my_bias=self.bias,
                     my_tensor=input_data,ker_size=3,input_chanels=2):
                Yijk=0
                for m in range(input_chanels):
                    for kh in range(ker_size):
                        for kw in range(ker_size):
                            Yijk+=my_kernel[kh][kw][m][c]*my_tensor[num][i-2+kh][j-2+kw][m]
                Yijk+=my_bias[c]
                return Yijk
            my_list=[]
            for num in range(self.output_channels):
                my_list_i=[]
                for i in range(start_y,stop_y,self.stride):
                    my_list_j=[]
                    for j in range(start_x,stop_x,self.stride):
                        my_list_c=[]
                        for c in range(self.output_channels):
                            Ynumijk=conv(num,c,i,j,my_kernel=self.kernel,my_bias=self.bias,
                                         my_tensor=input_data,ker_size=self.kernel_size,input_channels=self.input_channels)
                            my_list_c.append(Ynumijk)
                        my_list_j.append(my_list_c)
                    my_list_i.append(my_list_j)
                my_list.append(my_list_i)
            my_tensor_final=np.array(my_list)
            return my_tensor_final
        else:
            _,_,h,w=input_data.shape
        out = np.empty([])
        return out

    def forward(self, input_data):
        pass
    def grad_x(self):
        pass
    def grad_kernel(self):
        pass

In [6]:
class Conv2DTrLayer(Layer):
    def __init__(self, kernel_size=3, input_channels=2, output_channels=3,
                 padding=0, stride=1, K_init=None, b_init=None):
        # padding: число (сколько отрезать от модифицированной входной карты)
        # Работаем с квадратными ядрами, поэтому kernel_size - одно число
        # stride - одно число (коэффициент расширения)
        # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
        self.name = 'Conv2DTr'
        self.kernel_size = kernel_size
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.kernel = K_init
        self.bias = b_init
        self.padding = padding
        self.stride = stride
    def forward(self, input_data):
        # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
        # Вначале нужно проверить на согласование размерностей входных данных и ядра!
        # Нужно заполнить Numpy-тензор out
        out = np.empty([])
        return out

    def forward(self, input_data):
        pass
    def grad_x(self):
        pass
    def grad_kernel(self):
        pass

#### 1.4 Теперь настало время теста.
#### Если вы всё сделали правильно, то запустив следующие ячейки у вас должна появиться надпись: Test PASSED

Переходить к дальнейшим заданиям не имеем никакого смысла, пока вы не добьётесь прохождение теста
    

In [7]:
! pip install keras

#### Чтение данных

In [8]:
import numpy as np
np.random.seed(123)  # for reproducibility
# from keras.utils import np_utils
import keras
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


Y_train = keras.utils.to_categorical(y_train, 10)
Y_test = keras.utils.to_categorical(y_test, 10)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(60000, 1, 28, 28) (60000, 10) (10000, 1, 28, 28) (10000, 10)


#### Подготовка моделей

In [9]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, Conv2D, MaxPooling2D

print(keras.__version__)

def get_keras_model():
    input_image = Input(shape=(1, 28, 28))
    pool1 = MaxPooling2D(pool_size=(2,2), data_format='channels_first')(input_image)
    flatten = Flatten()(pool1)
    dense1 = Dense(10, activation='softmax')(flatten)
    model = Model(inputs=input_image, outputs=dense1)

    from keras.optimizers import Adam, SGD
    sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    history = model.fit(X_train, Y_train, validation_split=0.25,
                        batch_size=32, epochs=2, verbose=1)
    return model

3.8.0


In [10]:
def get_our_model(keras_model):
    maxpool = MaxPooling()
    flatten = FlattenLayer()
    input_image = Input(shape=(1, 28, 28))
    model = Sequential([
    MaxPooling2D(pool_size=(2,2), data_format='channels_first', input_shape=(1, 28, 28))])
    dense = DenseLayer(196, 10, W_init=keras_model.get_weights()[0],
                       b_init=keras_model.get_weights()[1])
    softmax = Softmax()
    net = Network([maxpool,flatten, dense, softmax])
    return net

In [11]:
keras_model = get_keras_model()
our_model = get_our_model(keras_model)

Epoch 1/2
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7731 - loss: 0.8455 - val_accuracy: 0.8927 - val_loss: 0.3807
Epoch 2/2
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8883 - loss: 0.3888 - val_accuracy: 0.9019 - val_loss: 0.3427


/usr/local/lib/python3.11/dist-packages/keras/src/layers/pooling/base_pooling.py:23: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(name=name, **kwargs)


In [12]:
keras_prediction = keras_model.predict(X_test)
our_model_prediction = our_model.predict(X_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [13]:

if np.sum(np.abs(keras_prediction - our_model_prediction)) < 0.01:
    print('Test PASSED')
else:
    print('Something went wrong!')

Test PASSED


### 2. Вычисление производных по входу для слоёв нейронной сети

В данном задании запрещено использовать численные формулы для вычисления производных.

#### 2.1  Реализуйте метод forward для класса CrossEntropy
Напоминание: $$ crossentropy = L(p, y) =  - \sum\limits_i y_i log p_i, $$
где вектор $(p_1, ..., p_k) $ -  выход классификационного алгоритма, а $(y_1,..., y_k)$ - правильные метки класса в унарной кодировке (one-hot encoding)

In [14]:
class CrossEntropy(object):
    def __init__(self, eps=0.00001):
        self.name = 'CrossEntropy'
        self.eps = eps

    def forward(self, input_data, labels):
        out=0
        for i,data in enumerate(input_data):
            out-=labels[i] @ np.log(data + self.eps).T
        return out


    def calculate_loss(self,input_data, labels):
        return self.forward(input_data, labels)

    def grad_x(self, input_data, lables):
        # print(input_data.shape,lables.shape)
        out=np.empty(input_data.shape[1])
        for i,data in enumerate(input_data):
            for j,elem in enumerate(data):
                # print(elem)
                out[j]-=lables[i,j]/(elem+self.eps)
        return out

#### 2.2  Реализуйте метод grad_x класса CrossEntropy, который возвращает $\frac{\partial L}{\partial p}$

Проверить работоспособность кода поможет следующий тест:

In [15]:
def numerical_diff_net(net, x, labels):
    eps = 0.00001
    right_answer = []
    for i in range(len(x[0])):
        delta = np.zeros(len(x[0]))
        delta[i] = eps
        diff = (net.calculate_loss(x + delta, labels) - net.calculate_loss(x-delta, labels)) / (2*eps)
        right_answer.append(diff)
    return np.array(right_answer).T

def test_net(net):
    x = np.array([[1, 2, 3], [2, 3, 4]])
    labels = np.array([[0.3, 0.2, 0.5], [0.3, 0.2, 0.5]])
    num_grad = numerical_diff_net(net, x, labels)
    grad = net.grad_x(x, labels)
    print(grad.shape)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)
loss = CrossEntropy()
test_net(loss)

(3,)
Test PASSED


#### 2.3  Реализуйте метод grad_x класса Softmax, который возвращает $\frac{\partial Softmax}{\partial x}$

Проверить работоспособность кода поможет следующий тест:

In [16]:
def numerical_diff_layer(layer, x):
    eps = 0.00001
    right_answer = []
    for i in range(len(x[0])):
        delta = np.zeros(len(x[0]))
        delta[i] = eps
        diff = (layer.forward(x + delta) - layer.forward(x-delta)) / (2*eps)
        right_answer.append(diff.T)
    return np.array(right_answer).T

def test_layer(layer):
    x = np.array([[1, 2, 3], [2, -3, 4]])
    num_grad = numerical_diff_layer(layer, x)
    print(num_grad.shape)
    grad = layer.grad_x(x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)

layer = Softmax()
test_layer(layer)

(2, 3, 3)
Test PASSED


#### 2.4  Реализуйте метод grad_x для классов ReLU и DenseLayer

In [17]:
layer = ReLU()
test_layer(layer)

(2, 3, 3)
Test PASSED


In [18]:
layer = DenseLayer(3,4)
test_layer(layer)

(2, 4, 3)
Test PASSED


#### 2.5 (4 балла) Для класса Network реализуйте метод grad_x, который должен реализовывать взятие производной от лосса по входу

In [39]:
net = Network([ReLU(), Softmax()], loss=CrossEntropy())
test_net(net)

(3,)
Test PASSED


In [19]:
net = Network([DenseLayer(3,784),ReLU(),DenseLayer(784,784),ReLU(),DenseLayer(784,256),ReLU(),DenseLayer(256,3), Softmax()], loss=CrossEntropy())
test_net(net)

(3,)
Test PASSED


### 3. Реализация градиентов по параметрам и метода обратного распространения ошибки с обновлением парметров сети

#### 3.1  Реализуйте функции grad_b и grad_W. При подготовке теста grad_W предполагается, что W является отномерным вектором.

In [20]:
def numerical_grad_b(input_size, output_size, b, W, x):
    eps = 0.00001
    right_answer = []
    for i in range(len(b)):
        delta = np.zeros(b.shape)
        delta[i] = eps
        dense1 = DenseLayer(input_size, output_size, W_init=W, b_init=b+delta)
        dense2 = DenseLayer(input_size, output_size, W_init=W, b_init=b-delta)
        diff = (dense1.forward(x) - dense2.forward(x)) / (2*eps)
        right_answer.append(diff.T)
    return np.array(right_answer).T

def test_grad_b():
    input_size = 3
    output_size = 4
    W_init = np.random.random((input_size, output_size))
    b_init = np.random.random((output_size,))
    x = np.random.random((2, input_size))

    dense = DenseLayer(input_size, output_size, W_init, b_init)
    grad = dense.grad_b(x)

    num_grad = numerical_grad_b(input_size, output_size, b_init, W_init, x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)

test_grad_b()

Test PASSED


In [21]:
def numerical_grad_W(input_size, output_size, b, W, x):
    eps = 0.00001
    right_answer = []
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            delta = np.zeros(W.shape)
            delta[i, j] = eps
            dense1 = DenseLayer(input_size, output_size, W_init=W+delta, b_init=b)
            dense2 = DenseLayer(input_size, output_size, W_init=W-delta, b_init=b)
            diff = (dense1.forward(x) - dense2.forward(x)) / (2*eps)
            right_answer.append(diff.T)
    return np.array(right_answer).T

def test_grad_W():
    input_size = 3
    output_size = 4
    W_init = np.random.random((input_size, output_size))
    b_init = np.random.random((4,))
    x = np.random.random((2, input_size))

    dense = DenseLayer(input_size, output_size, W_init, b_init)
    grad = dense.grad_W(x)

    num_grad = numerical_grad_W(input_size, output_size, b_init, W_init, x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)

test_grad_W()

Test PASSED


#### 3.2 Полностью реализуйте метод обратного распространения ошибки в функции train_step класса Network


Рекомендуем реализовать сначала функцию Network.grad_param(), которая возвращает список длиной в количество слоёв и элементом которого является список градиентов по параметрам.
После чего, имея список градиентов, написать функцию обновления параметров для каждого слоя.

Совет: рекомендуем написать тест для кода подсчета градиента по параметрам, чтобы быть уверенным в том, что градиент через всю сеть считается правильно
    

#### 3.3 Ознакомьтесь с реализацией функции fit класса Network. Запустите обучение модели. Если всё работает правильно, то точность на валидации должна будет возрастать

In [32]:
net = Network([DenseLayer(784, 10), Softmax()], loss=CrossEntropy())
# net = Network([dense1], loss=CrossEntropy())
trainX = X_train.reshape(len(X_train), -1)
net.fit(trainX, Y_train, validation_split=0.25,
            batch_size=2, nb_epoch=3, learning_rate=0.01)

100%|██████████| 22500/22500 [00:31<00:00, 721.57it/s]


1 epoch: val 0.91


100%|██████████| 22500/22500 [00:27<00:00, 831.63it/s]


2 epoch: val 0.91


100%|██████████| 22500/22500 [00:27<00:00, 824.83it/s]


3 epoch: val 0.92


In [30]:
from sklearn.metrics import accuracy_score
testX = X_test.reshape(len(X_test), -1)
pred=net.predict(testX)
pred=np.argmax(pred, axis=1)
Y_test=np.argmax(Y_test, axis=1)
# print(*pred)
print(accuracy_score(pred,Y_test))

0.908


In [ ]:
net = Network([DenseLayer(784, 64),ReLU(), DenseLayer(64, 10), Softmax()], loss=CrossEntropy())
trainX = X_train.reshape(len(X_train), -1)
# net.forward(trainX)
net.fit(trainX[::10], Y_train[::10], validation_split=0.25,
            batch_size=5, nb_epoch=10, learning_rate=0.01)

100%|██████████| 900/900 [07:36<00:00,  1.97it/s]


1 epoch: val 0.22


100%|██████████| 900/900 [07:19<00:00,  2.05it/s]


2 epoch: val 0.21


 23%|██▎       | 206/900 [01:38<06:55,  1.67it/s]

#### 3.5 Продемонстрируйте, что ваша реализация позволяет обучать более глубокие нейронные сети

Обучаю на малом количестве данных, так как обучение требует много времени

In [27]:
net = Network([DenseLayer(784, 32), ReLU(),DenseLayer(32, 16), ReLU(), DenseLayer(16, 10), Softmax()], loss=CrossEntropy())
trainX = X_train.reshape(len(X_train), -1)
# net.forward(trainX)
net.fit(trainX[::6], Y_train[::6], validation_split=0.25,
            batch_size=8, nb_epoch=20, learning_rate=0.05)

100%|██████████| 937/937 [06:39<00:00,  2.34it/s]


1 epoch: val 0.11


 10%|█         | 98/937 [00:30<04:16,  3.27it/s]


KeyboardInterrupt: 

In [ ]:
pred=net.predict(testX)
pred=np.argmax(pred, axis=1)

# print(*pred)
print(accuracy_score(pred,Y_test))

0.1925
